In [2]:
%pip install beautifulsoup4

     -------------------------------------- 147.9/147.9 KB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Lattae\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
%pip install flask

     -------------------------------------- 101.7/101.7 KB 1.5 MB/s eta 0:00:00
  Using cached werkzeug-3.0.3-py3-none-any.whl (227 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Lattae\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [9]:
from flask import Flask, jsonify, request
import json
import requests
from bs4 import BeautifulSoup
import re

In [10]:
app = Flask(__name__)

In [12]:
def clean_text(text):
    # Replace newline (\n), carriage return (\r), and multiple spaces with a single space
    return re.sub(r'[\n\r]+', ' ', text).strip()

In [16]:
def extract_website_info(url):
    try:
        # Send request to website
        response = requests.get(url)
        response.raise_for_status()  # Raise error for bad response
        
        # Parse HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract Judul
        judul = soup.find('h1').text.strip() if soup.find('h1') else '-'
        judul = clean_text(judul)
        
        def get_table_data(header_name):
            header = soup.find('th', string=header_name)
            if header:
                td = header.find_next_sibling('td')
                if td:
                    link = td.find('a')
                    if link:
                        # Extract the href attribute of the <a> tag
                        return 'https://peraturan.go.id' + link.get('href')
                    else:
                        # Return the text content of the table cell
                        return clean_text(td.text.strip())
            return '-'
        
        # Extracting specific data from the table
        jenis_bentuk_peraturan = get_table_data('Jenis/Bentuk Peraturan')
        pemrakarsa = get_table_data('Pemrakarsa')
        nomor = get_table_data('Nomor')
        tahun = get_table_data('Tahun')
        tentang = get_table_data('Tentang')
        tempat_penetapan = get_table_data('Tempat Penetapan')
        tanggal_penetapan = get_table_data('Ditetapkan Tanggal')
        pejabat_menetapkan = get_table_data('Pejabat yang Menetapkan')
        status = get_table_data('Status')
        dokumen = get_table_data('Dokumen Peraturan')  # Update header name here
        tahun_pengundangan = get_table_data('Tahun Pengundangan')
        no_pengundangan = get_table_data('Nomor Pengundangan')
        no_tambahan = get_table_data('Nomor Tambahan')
        tanggal_pengundangan = get_table_data('Tanggal Pengundangan')
        pejabat_pengundangan = get_table_data('Pejabat Pengundangan')
        
        # Extract mengubah and dasar hukum information list
        mengubah = []
        dasar_hukum = []
        
        div_card_body = soup.find_all('div', class_='card-body')
        
        # Print the index of each div.card-body
        for index, section in enumerate(div_card_body):
            if 'Mengubah :' in section.text:
                for item in section.find_all('li'):
                    link = item.find('a')
                    if link:
                        mengubah.append({
                            'text': clean_text(link.text.strip()),
                            'link': link.get('href')
                        })
                    else:
                        mengubah.append({'text': clean_text(item.text.strip()), 'link': None})
        
        # Extract the second 'ul' element within the second 'div.card-body'
        if len(div_card_body) > 1:
            sec_div_card_body = div_card_body[2]
            ul_tags = sec_div_card_body.find_all('ul')
            if ul_tags:
                sec_ul_tag = ul_tags[0]  # Second ul in the second div
                li_tags = sec_ul_tag.find_all('li')
                for item in li_tags:
                    link = item.find('a')
                    if link:
                        dasar_hukum.append({
                            'text': clean_text(item.text.strip()),
                            'link': link.get('href')
                        })
                    else:
                        dasar_hukum.append({'text': clean_text(item.text.strip()), 'link': None})
        
        data = {
            'judul': judul,
            'jenis_bentuk_peraturan': jenis_bentuk_peraturan,
            'pemrakarsa': pemrakarsa,
            'nomor': nomor,
            'tahun': tahun,
            'tentang': tentang,
            'tempat_penetapan': tempat_penetapan,
            'tanggal_penetapan': tanggal_penetapan,
            'pejabat_menetapkan': pejabat_menetapkan,
            'status': status,
            'dokumen': dokumen,
            'tahun_pengundangan': tahun_pengundangan,
            'no_pengundangan': no_pengundangan,
            'no_tambahan': no_tambahan,
            'tanggal_pengundangan': tanggal_pengundangan,
            'pejabat_pengundangan': pejabat_pengundangan,
            'mengubah': mengubah,
            'dasar_hukum': dasar_hukum
        }
        
        return data
        
    except requests.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return None

In [17]:
def post_data_to_api(api_url, data):
    try:
        headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        }
        response = requests.post(api_url, json=data, headers=headers)
        response.raise_for_status()
        print('Data successfully posted:', response.status_code)
        print('Response:', response.json())
    except requests.RequestException as e:
        print(f"Error posting data to the API: {e}")

In [18]:
if __name__ == '__main__':
    url = "https://www.peraturan.go.id/id/perppu-no-1-tahun-2022"
    extracted_data = extract_website_info(url)
    
    if extracted_data:
        print("Extracted Data:")
        print(json.dumps(extracted_data, ensure_ascii=False, indent=4))
        
        api_url = "http://127.0.0.1:3000/api/peraturan"
        post_data_to_api(api_url, extracted_data)

Extracted Data:
{
    "judul": "Peraturan Pemerintah Pengganti Undang-undang Nomor 1 Tahun 2022 Tentang Perubahan Atas Undang-undang Nomor 7 Tahun 2017 Tentang pemilihan Umum",
    "jenis_bentuk_peraturan": "PERATURAN PEMERINTAH PENGGANTI UNDANG-UNDANG",
    "pemrakarsa": "PEMERINTAH PUSAT",
    "nomor": "1",
    "tahun": "2022",
    "tentang": "PERUBAHAN ATAS UNDANG-UNDANG NOMOR 7 TAHUN 2017 TENTANG PEMILIHAN UMUM",
    "tempat_penetapan": "Jakarta",
    "tanggal_penetapan": "12 Desember 2022",
    "pejabat_menetapkan": "",
    "status": "Berlaku",
    "dokumen": "https://peraturan.go.id/files/Salinan+Perpu+Nomor+1+tahun+2022.pdf",
    "tahun_pengundangan": "2022",
    "no_pengundangan": "224",
    "no_tambahan": "6832",
    "tanggal_pengundangan": "12 Desember 2022",
    "pejabat_pengundangan": "",
    "mengubah": [
        {
            "text": "Undang-Undang Nomor 7 Tahun 2017",
            "link": "/id/uu-no-7-tahun-2017"
        }
    ],
    "dasar_hukum": [
        {
           

In [4]:
# Extract Web Content
url = "https://www.peraturan.go.id/id/perppu-no-1-tahun-2022"
json_data  = extract_website_info(url)

C:\Users\Lattae\AppData\Local\Temp\ipykernel_3884\316193528.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  header = soup.find('th', text=header_name)


In [5]:
if json_data:
    print(json_data)

{
    "judul": "Peraturan Pemerintah Pengganti Undang-undang Nomor 1 Tahun 2022 Tentang Perubahan Atas Undang-undang Nomor 7 Tahun 2017 Tentang pemilihan Umum",
    "jenis_bentuk_peraturan": "PERATURAN PEMERINTAH PENGGANTI UNDANG-UNDANG",
    "pemrakarsa": "PEMERINTAH PUSAT",
    "nomor": "1",
    "tahun": "2022",
    "tentang": "PERUBAHAN ATAS UNDANG-UNDANG NOMOR 7 TAHUN 2017 TENTANG PEMILIHAN UMUM",
    "tempat_penetapan": "Jakarta",
    "tanggal_penetapan": "12 Desember 2022",
    "pejabat_menetapkan": "",
    "status": "Berlaku",
    "dokumen": "https://peraturan.go.id/files/Salinan+Perpu+Nomor+1+tahun+2022.pdf",
    "tahun_pengundangan": "2022",
    "no_pengundangan": "224",
    "no_tambahan": "6832",
    "tanggal_pengundangan": "12 Desember 2022",
    "pejabat_pengundangan": "",
    "mengubah": [
        {
            "text": "Undang-Undang Nomor 7 Tahun 2017",
            "link": "/id/uu-no-7-tahun-2017"
        }
    ],
    "dasar_hukum": [
        {
            "text": "Tentan